In [1]:
#Try LDA again with less topics, try 15 this time:
#Try out hierarchical LDA with GENSIM to get the optimal number of topics
import os
from os import listdir
from os.path import isfile, join
from gensim import corpora, models, similarities
#import DocIterator as DocIt
import codecs
import logging
import gensim
from collections import namedtuple
from gensim.utils import smart_open, simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim.models.hdpmodel import HdpModel
from gensim.models.ldamodel import LdaModel
import itertools

#Log in thing that Gensim needs for some reason
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
logging.root.level = logging.INFO
#Document path (no latex, still has stop words though! Need to clean these out.)
myDirPath = '/Users/daniel/Desktop/ArxivProject/TfIdfPredict/Lda/GensimLDA/CleanedAbstractsNoLatexNoStopwords'
#Now get out the half documents:
#myDirPath = '/Users/daniel/Desktop/ArxivProject/TfIdfPredict/Lda/GensimLDA/CleanedAbstractsNoLatexNoStopwordsHALF'

In [2]:
def tokenize(text):
    return [token for token in simple_preprocess(text) if token not in STOPWORDS]

In [3]:
docLabels = []
docLabels = [f for f in listdir(myDirPath) if f.endswith('.txt')]

data = []
for doc in docLabels:
    abst = codecs.open(myDirPath +'/'+doc,'r','utf-8')
    #abst = open(myDirPath +'/'+doc,'r').encode('utf-8')
    #data.append(abst)
    data.append(abst.read())
    abst.close()

In [4]:
#Setup a document stream which is a list of lists of tokens for each document. This will used build the dictonary
doc_stream = []
for abst in data:
    toks = tokenize(abst)
    doc_stream.append(toks)



In [5]:
def iter_documents(top_directory):
    """Iterate over all documents, yielding a document (=list of utf8 tokens) at a time."""
    for root, dirs, files in os.walk(top_directory):
        for file in filter(lambda file: file.endswith('.txt'), files):
            document = open(os.path.join(root, file)).read() # read the entire document, as one big string
            yield gensim.utils.tokenize(document, lower=True) # or whatever tokenization suits you

class MyCorpus(object):
    def __init__(self, top_dir):
        self.top_dir = top_dir
        self.dictionary = gensim.corpora.Dictionary(iter_documents(top_dir))
        self.dictionary.filter_extremes(no_below=5, keep_n=30000) # check API docs for pruning params
        #self.dictionary.filter_extremes(no_below=20, no_above=0.1)
        
    def __iter__(self):
        for tokens in iter_documents(self.top_dir):
            yield self.dictionary.doc2bow(tokens)

corpus = MyCorpus(myDirPath) # create a dictionary
#for vector in corpus: # convert each document to a bag-of-word vector
    #print(vector)

INFO:gensim.corpora.dictionary:adding document #0 to Dictionary(0 unique tokens: [])
INFO:gensim.corpora.dictionary:adding document #10000 to Dictionary(14147 unique tokens: ['cones', 'bun', 'quasistatic', 'phonons', 'octonions']...)
INFO:gensim.corpora.dictionary:adding document #20000 to Dictionary(19257 unique tokens: ['cones', 'bun', 'glance', 'quasistatic', 'phonons']...)
INFO:gensim.corpora.dictionary:adding document #30000 to Dictionary(22990 unique tokens: ['quasistatic', 'phonons', 'dropped', 'bilinears', 'gravitons']...)
INFO:gensim.corpora.dictionary:adding document #40000 to Dictionary(25952 unique tokens: ['quasistatic', 'phonons', 'dropped', 'kirwan', 'bilinears']...)
INFO:gensim.corpora.dictionary:built Dictionary(28699 unique tokens: ['quasistatic', 'phonons', 'wcfts', 'dropped', 'kirwan']...) from 49962 documents (total 1797183 corpus positions)
INFO:gensim.corpora.dictionary:discarding 18064 tokens: [('hydr', 2), ('schwarzschi', 3), ('gravexcitons', 1), ('belinskii', 

In [6]:
dictionary = corpus.dictionary

In [7]:
dictionary.save('AbstractDictPrunedHALF.dict')

INFO:gensim.utils:saving Dictionary object under AbstractDictPrunedHALF.dict, separately None


In [8]:
corpus = [dictionary.doc2bow(text) for text in doc_stream]

In [9]:
corpora.MmCorpus.serialize('AbstractsPrunedHALF.mm', corpus)

INFO:gensim.corpora.mmcorpus:storing corpus in Matrix Market format to AbstractsPrunedHALF.mm
INFO:gensim.matutils:saving sparse matrix to AbstractsPrunedHALF.mm
INFO:gensim.matutils:PROGRESS: saving document #0
INFO:gensim.matutils:PROGRESS: saving document #1000
INFO:gensim.matutils:PROGRESS: saving document #2000
INFO:gensim.matutils:PROGRESS: saving document #3000
INFO:gensim.matutils:PROGRESS: saving document #4000
INFO:gensim.matutils:PROGRESS: saving document #5000
INFO:gensim.matutils:PROGRESS: saving document #6000
INFO:gensim.matutils:PROGRESS: saving document #7000
INFO:gensim.matutils:PROGRESS: saving document #8000
INFO:gensim.matutils:PROGRESS: saving document #9000
INFO:gensim.matutils:PROGRESS: saving document #10000
INFO:gensim.matutils:PROGRESS: saving document #11000
INFO:gensim.matutils:PROGRESS: saving document #12000
INFO:gensim.matutils:PROGRESS: saving document #13000
INFO:gensim.matutils:PROGRESS: saving document #14000
INFO:gensim.matutils:PROGRESS: saving doc

In [10]:
%time lda = LdaModel(corpus,num_topics=15,id2word=dictionary,passes=30)

INFO:gensim.models.ldamodel:using symmetric alpha at 0.06666666666666667
INFO:gensim.models.ldamodel:using serial LDA version on this node
INFO:gensim.models.ldamodel:running online LDA training, 15 topics, 30 passes over the supplied corpus of 49962 documents, updating model once every 2000 documents, evaluating perplexity every 20000 documents, iterating 50x with a convergence threshold of 0.001000
INFO:gensim.models.ldamodel:PROGRESS: pass 0, at document #2000/49962
INFO:gensim.models.ldamodel:merging changes from 2000 documents into a model of 49962 documents
INFO:gensim.models.ldamodel:topic #13 (0.067): 0.019*theory + 0.014*gauge + 0.012*theories + 0.010*dimensional + 0.009*field + 0.009*quantum + 0.009*non + 0.008*space + 0.007*model + 0.007*time
INFO:gensim.models.ldamodel:topic #4 (0.067): 0.018*field + 0.011*model + 0.010*scalar + 0.009*energy + 0.008*theory + 0.007*gauge + 0.006*black + 0.006*effective + 0.006*spacetime + 0.006*non
INFO:gensim.models.ldamodel:topic #14 (0.06

CPU times: user 19min 8s, sys: 3.31 s, total: 19min 11s
Wall time: 19min 19s


In [11]:
lda.print_topics(num_topics=15, num_words=10)

INFO:gensim.models.ldamodel:topic #0 (0.067): 0.017*singularity + 0.016*defects + 0.016*singularities + 0.012*big + 0.012*parity + 0.011*neutrino + 0.011*singular + 0.010*defect + 0.010*problem + 0.009*bose
INFO:gensim.models.ldamodel:topic #1 (0.067): 0.040*phase + 0.029*quantum + 0.028*states + 0.025*temperature + 0.024*state + 0.021*critical + 0.018*finite + 0.018*transition + 0.014*study + 0.013*systems
INFO:gensim.models.ldamodel:topic #2 (0.067): 0.051*string + 0.040*theory + 0.028*brane + 0.021*branes + 0.018*type + 0.016*dimensional + 0.013*ads + 0.013*study + 0.012*supergravity + 0.012*non
INFO:gensim.models.ldamodel:topic #3 (0.067): 0.078*black + 0.045*hole + 0.034*ads + 0.027*holes + 0.023*horizon + 0.013*cft + 0.011*charged + 0.011*holographic + 0.011*asymptotically + 0.010*near
INFO:gensim.models.ldamodel:topic #4 (0.067): 0.069*field + 0.051*scalar + 0.019*potential + 0.018*fields + 0.017*inflation + 0.014*magnetic + 0.014*non + 0.011*vacuum + 0.011*modes + 0.010*tensor


['0.017*singularity + 0.016*defects + 0.016*singularities + 0.012*big + 0.012*parity + 0.011*neutrino + 0.011*singular + 0.010*defect + 0.010*problem + 0.009*bose',
 '0.040*phase + 0.029*quantum + 0.028*states + 0.025*temperature + 0.024*state + 0.021*critical + 0.018*finite + 0.018*transition + 0.014*study + 0.013*systems',
 '0.051*string + 0.040*theory + 0.028*brane + 0.021*branes + 0.018*type + 0.016*dimensional + 0.013*ads + 0.013*study + 0.012*supergravity + 0.012*non',
 '0.078*black + 0.045*hole + 0.034*ads + 0.027*holes + 0.023*horizon + 0.013*cft + 0.011*charged + 0.011*holographic + 0.011*asymptotically + 0.010*near',
 '0.069*field + 0.051*scalar + 0.019*potential + 0.018*fields + 0.017*inflation + 0.014*magnetic + 0.014*non + 0.011*vacuum + 0.011*modes + 0.010*tensor',
 '0.066*quantum + 0.031*theory + 0.020*gravity + 0.017*field + 0.013*theories + 0.012*review + 0.012*recent + 0.012*physics + 0.011*classical + 0.010*general',
 '0.041*space + 0.031*time + 0.016*field + 0.013*s

In [12]:
#These are awesome! Let's save the model for later:
lda.save('lda15TopicsHALF.model')

INFO:gensim.utils:saving LdaState object under lda15TopicsHALF.model.state, separately None
INFO:gensim.utils:saving LdaModel object under lda15TopicsHALF.model, separately None
INFO:gensim.utils:not storing attribute dispatcher
INFO:gensim.utils:not storing attribute state


In [13]:
lda.log_perplexity(corpus)
#Notice this stays pretty constant compared to 19 topics, so I think we've hit our best case scenario...

INFO:gensim.models.ldamodel:-7.553 per-word bound, 187.8 perplexity estimate based on a held-out corpus of 49962 documents with 1640557 words


-7.5529268205475537

In [14]:
import pyLDAvis
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

In [15]:
pyLDAvis.gensim.prepare(lda, corpus, dictionary)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
9      10.084447        1       1  0.075153 -0.002686
13      8.969397        1       2  0.104193  0.003053
12      8.848332        1       3  0.124886 -0.042162
10      8.631418        1       4  0.138527  0.124860
6       7.902751        1       5  0.062163  0.012854
4       7.530669        1       6 -0.005013 -0.201015
11      7.452128        1       7 -0.099096 -0.186030
2       6.973563        1       8  0.100859  0.095446
5       6.620145        1       9  0.032579  0.045929
3       6.221383        1      10  0.001499 -0.029103
14      5.053670        1      11 -0.094792 -0.174957
1       4.987795        1      12 -0.103110 -0.136597
7       4.719714        1      13  0.058780  0.074026
8       4.083722        1      14  0.038918  0.216803
0       1.920864        1      15 -0.435546  0.199579, topic_info=      Category          Freq           Term  Total  loglift  logprob
3279   Default   8135.000000          black   8135  30.0000  30.0000
1308   Default  10910.000000        quantum  10910  29.0000  29.0000
7402   Default  18169.000000          field  18169  28.0000  28.0000
4266   Default   6319.000000         string   6319  27.0000  27.0000
2471   Default   8037.000000         scalar   8037  26.0000  26.0000
1013   Default  11159.000000          gauge  11159  25.0000  25.0000
4213   Default  20751.000000         theory  20751  24.0000  24.0000
5256   Default   9037.000000        gravity   9037  23.0000  23.0000
792    Default   9139.000000          space   9139  22.0000  22.0000
8019   Default   4645.000000           hole   4645  21.0000  21.0000
7807   Default  12311.000000          model  12311  20.0000  20.0000
2414   Default   5733.000000       symmetry   5733  19.0000  19.0000
6155   Default   5091.000000            ads   5091  18.0000  18.0000
6378   Default   5835.000000           time   5835  17.0000  17.0000
10380  Default   3694.000000          phase   3694  16.0000  16.0000
3524   Default   4075.000000       function   4075  15.0000  15.0000
3918   Default   3792.000000          brane   3792  14.0000  14.0000
2674   Default   6173.000000      solutions   6173  13.0000  13.0000
9056   Default   4904.000000      equations   4904  12.0000  12.0000
2476   Default   3398.000000       boundary   3398  11.0000  11.0000
4788   Default   3968.000000   cosmological   3968  10.0000  10.0000
2560   Default   8103.000000       theories   8103   9.0000   9.0000
9529   Default   3806.000000           spin   3806   8.0000   8.0000
258    Default   2739.000000          holes   2739   7.0000   7.0000
8874   Default   2795.000000         branes   2795   6.0000   6.0000
2004   Default   3732.000000           loop   3732   5.0000   5.0000
9449   Default   8034.000000         energy   8034   4.0000   4.0000
1167   Default   2815.000000        algebra   2815   3.0000   3.0000
4975   Default   2692.000000       universe   2692   2.0000   2.0000
10361  Default   2474.000000        horizon   2474   1.0000   1.0000
...        ...           ...            ...    ...      ...      ...
2369   Topic15    154.204549        network    155   4.0660  -5.2012
4159   Topic15    361.059915       neutrino    362   3.9015  -4.5175
9284   Topic15    110.254346       homotopy    111   4.1290  -5.4720
3420   Topic15    258.102932          ideas    259   3.9474  -4.8063
3129   Topic15    212.138105           rule    213   3.9868  -4.9624
3373   Topic15    150.201040       reversal    151   4.0614  -5.2319
7254   Topic15    155.190354      difficult    156   4.0482  -5.2125
5413   Topic15    256.075785         exotic    257   3.9175  -4.8440
9919   Topic15    108.246120       aharonov    109   4.1178  -5.5014
4842   Topic15    125.210985        reveals    126   4.0730  -5.4013
2043   Topic15    498.344473    singularity    856   3.4743  -4.0839
10457  Topic15    368.188340         parity    506   3.6352  -4.4488
729

In [ ]:
#Fantastic.  Now we can test our corpus against these topics, get a distribution for each as a function of time and do the time series 
#properly!